In [29]:
import xml.etree.ElementTree as ET

## MARC XML
Ziel: finde Entsprechungen des MARC mit dem DataCite Schema und speichere in Variable.

Damit bei Erstellen der neuen xml mit "varable".find das Subfeld und der Text zugewiesen werden können

zum Bleistift:
* dataCite xml: creator = MARC datafield tag = 100
* als Variable "author" speichern
* beim xml erstellen kann auf Subfield Code "a" zugegriffen werden

In [30]:
# load Marc xml
tree = ET.parse("exportExample.xml")
collection = tree.getroot() # rename root in collection (export starts with collection)

In [31]:
records = collection[0] # first record in collection

identifierMRC = records.findall(".//datafield[@tag='024']")
identifierMRC

identifierDict = {}

for identifier in identifierMRC:
    identifierDict[identifier.find("subfield[@code='2']").text] = identifier.find("subfield[@code='a']").text
    print(identifier.find("subfield[@code='2']").text)
    print(identifier.find("subfield[@code='a']").text)

identifierDict

doi
10.35011/risc.22-04
urn
urn:nbn:at:at-ubl:3-12218


{'doi': '10.35011/risc.22-04', 'urn': 'urn:nbn:at:at-ubl:3-12218'}

In [32]:
records = collection[0] # first record in collection

identifierMRC = records.find(".//datafield[@tag='024']")
identifierMRC.find("subfield[@code='a']").text

authorMRC = records.find(".//datafield[@tag='100']") # find field 100
authorMRC.find("subfield[@code='a']").text # find subfield "a" and get text
authorMRC.find("subfield[@code='a']").text.split(",") # split family/given Name
        # subfield a = creatorName
        # subfield a - split "," split[0] = familyName split[1] = givenName
        # subfield??? = nameIdentifier -- z.B. ORCID
        # subfield??? = affiliation


titleMRC = records.find(".//datafield[@tag='245']")
titleMRC.find("subfield[@code='a']").text
titleMRC.find("subfield[@code='b']").text

publisherMRC = records.find(".//datafield[@tag='264']")
publisherMRC.find("subfield[@code='b']").text
publisherMRC.find("subfield[@code='c']").text

languageMRC = records.find(".//datafield[@tag='041']")
languageMRC.find("subfield[@code='a']").text

'eng'

In [33]:
# NB! Für Wiederholende Felder muss .findall gewählt werden
records.findall(".//datafield[@tag='024']")

[<Element 'datafield' at 0x000001C0C13A6CF0>,
 <Element 'datafield' at 0x000001C0C13A6B60>]

## DataCite xml
Idee:

Mach mir ein komplettes DataCite xml

such mir den Text aus den records

In [34]:
dcOutput = ET.Element("resource")
dcOutput.attrib = {
    "xmlns:xsi":"http://www.w3.org/2001/XMLSchema-instance",
    "xsi:schemaLocation": "http://datacite.org/schema/kernel-4 https://schema.datacite.org/meta/kernel-4.4/metadata.xsd"
    }

# Identifier -------------------------------------------------------------

identifier = ET.Element("identifier")
identifier.attrib = {"identifierType":"DOI"}
identifier.text = identifierMRC.find("subfield[@code='a']").text
dcOutput.append(identifier)

# Creators -------------------------------------------------------------
creators = ET.Element("creators")
creator = ET.SubElement(creators, "creator")
creatorName = ET.SubElement(creator, "creatorName")
creatorName.attrib = {"nameType":"Personal"}
creatorName.text = authorMRC.find("subfield[@code='a']").text

givenName = ET.SubElement(creator, "givenName")
givenName.text = authorMRC.find("subfield[@code='a']").text.split(",")[1]

familyName = ET.SubElement(creator, "familyName")
familyName.text = authorMRC.find("subfield[@code='a']").text.split(",")[0]

nameIdentifier = ET.SubElement(creator, "nameIdentifier")
nameIdentifier.attrib = {
    "schemeURI":"https://orcid.org/",
    "nameIdentifierScheme":"ORCID"
    }
nameIdentifier.text = " "

affiliation = ET.SubElement(creator, "affiliation")
affiliation.text = " "

dcOutput.append(creators)

# Titles -------------------------------------------------------------
titles = ET.Element("titles")
title = ET.SubElement(titles, "title")
title.attrib = {
    "xml:lang":"en-US"
    }
title.text = titleMRC.find("subfield[@code='a']").text
subtitle = ET.SubElement(titles, "title")
subtitle.attrib = {
    "xml:lang":"en-US",
    "titleType":"Subtitle"
    }
subtitle.text = titleMRC.find("subfield[@code='b']").text

dcOutput.append(titles)

# publisher -------------------------------------------------------------
publisher = ET.Element("publisher")
publisher.attrib = {"xml:lang":"en"}
publisher.text = publisherMRC.find("subfield[@code='b']").text
dcOutput.append(publisher)

# publication Year -------------------------------------------------------------
publicationYear = ET.Element("publicationYear")
publicationYear.text = publisherMRC.find("subfield[@code='c']").text
dcOutput.append(publicationYear)

# Language -------------------------------------------------------------
language = ET.Element("language")
language.text = languageMRC.find("subfield[@code='a']").text
dcOutput.append(language)

# create Tree
dcTree = ET.ElementTree(dcOutput)
# write output-
dcTree.write("dcTest_draft2.xml")

In [35]:
# dcTree = ET.ElementTree(dcOutput)
# dcTree.write("dcTest.xml")